In [1]:
import numpy as np
import tables
from icecube import dataio
import glob
from icecube import icetray, dataio, dataclasses,simclasses,recclasses, gulliver, finiteReco, paraboloid
from I3Tray import *
from icecube.hdfwriter import I3SimHDFWriter
from icecube import millipede, linefit
from icecube.hdfwriter import I3HDFWriter
from icecube import astro
from icecube import NewNuFlux
import time
import operator
import matplotlib.pyplot as plt

RuntimeError: FATAL: module compiled as little endian, but detected different endianness at runtime

In [2]:
MC_file = tables.open_file('/data/user/xk35/BDT/merged_hdf/MC/3000file/mc_3000.hdf')
Data_test_file = tables.open_file('/data/user/xk35/BDT/merged_hdf/Data/test_file/data_test_24days.hdf')
Data_train_file = tables.open_file('/data/user/xk35/BDT/merged_hdf/Data/train_file/data_24days.hdf')

MC_CC = np.load('/data/user/xk35/BDT/Corner_Clipper/Charge_Ratio/MC/prob_cc.npy')
Data_train_CC = np.load('/data/user/xk35/BDT/Corner_Clipper/Charge_Ratio/Data/prob_cc_data_train.npy')
Data_test_CC = np.load('/data/user/xk35/BDT/Corner_Clipper/Charge_Ratio/Data_test/prob_cc_data_test.npy')

In [3]:
a = time.time()
gamma = 2.7
# save MC and Data
#livetime = good_run_list.sum()[0]
#livetime = good_run_list.sum()[0]

def delta_angle(pt1_zen, pt1_azi=None, pt2_zen=None, pt2_azi=None):


    z1=np.array(pt1_zen)
    z2=np.array(pt2_zen)
    a1=np.array(pt1_azi)
    a2=np.array(pt2_azi)
    pi = np.pi
    # haversine!
    cos_alpha = np.cos(z1-z2) - np.cos(pi/2.0 - z1)*np.cos(pi/2.0 - z2)*(1-np.cos(a1-a2))
    alpha=180/pi*np.arccos(cos_alpha)
    alpha[np.isnan(alpha)]=180
    return alpha

def SplitMinZenithFunc(zen1,zen2,zen3,zen4):
        zen1[(zen1<0)|(zen1!=zen1)]=400
        zen2[(zen2<0)|(zen2!=zen2)]=400
        zen3[(zen3<0)|(zen3!=zen3)]=400
        zen4[(zen4<0)|(zen4!=zen4)]=400
        return np.degrees(np.min(np.transpose((zen1,zen2,zen3,zen4)),axis=1))

BayesianFunc = lambda blogl,logl: np.nan_to_num(blogl - logl)


# Precuts

In [4]:
def GetCutIndices(file,CC_file,Cuts):
        MyIndices={}
        Indices=np.ones(len(file.root.SplineMPE),dtype=bool)
        for item in Cuts:
                
                if item[0]=='MPEFitParaboloidFitParams':
                        var=np.sqrt(file.root._f_get_child(item[0]).read()[item[1]]**2+file.root._f_get_child(item[0]).read()[item[2]]**2)/np.sqrt(2)
                        Indices=(Indices)&(item[3](var,item[4]))
                elif item[1]=='delta_angle':
                        tracks=item[0].split(',')
                        DA=delta_angle(file.root._f_get_child(tracks[0]).read()['zenith'],file.root._f_get_child(tracks[0]).read()['azimuth'],file.root._f_get_child(tracks[1]).read()['zenith'],file.root._f_get_child(tracks[1]).read()['azimuth'])
                        Indices=(Indices)&(item[2](DA,item[3]))
                elif item[0]=='CornerClipper':
                        Indices=(Indices)&(item[1](CC_file,item[2]))
                    
                else:
                        Indices=(Indices)&(item[2](file.root._f_get_child(item[0]).read()[item[1]],item[3]))
                        
                print(Indices.sum())
        MyIndices=Indices
        return MyIndices

In [5]:
GC_ra , GC_dec = astro.gal_to_equa(0., 0.)
zen_min = GC_dec-np.radians(10)+np.radians(90.)
zen_max = GC_dec+np.radians(10)+np.radians(90.)

Precuts=[]
#Precuts+=[('SplineMPE','zenith',operator.gt,zen_min)]
#Precuts+=[('SplineMPE','zenith',operator.le,zen_max)]
Precuts+=[('SplineMPE','fit_status',operator.eq,0)]
Precuts+=[('SplineMPEFitParams','rlogl',operator.lt,9)]
Precuts+=[('SplineMPEDirectHitsE','dir_track_length',operator.gt,250)]
Precuts+=[('MPEFitParaboloidFitParams','err1','err2',operator.lt,np.radians(4.5))]
Precuts+=[('SplineMPE_MillipedeHighEnergyMIEFitParams','exists',operator.eq,1)]
Precuts+=[('CornerClipper', operator.eq,0)]

In [6]:

a = MC_file.root.SplineMPE.cols.zenith[:]
GC_ra , GC_dec = astro.gal_to_equa(0., 0.)
mask1 = ((a-np.radians(90.))>GC_dec-np.radians(10)) & ((a-np.radians(90.))<GC_dec+np.radians(10))
len(a[mask1])

124375

In [7]:
print('MC')
MC_cut_indices = GetCutIndices(MC_file,MC_CC,Precuts)
print('Data test')
Data_test_cut_indices = GetCutIndices(Data_test_file,Data_test_CC,Precuts)
print('Data train')
Data_train_cut_indices = GetCutIndices(Data_train_file,Data_train_CC,Precuts)

MC
9031302
7556367
5640743
5174903
5174525
4689749
Data test
5795509
4461087
3145145
2953278
2953205
2630944
Data train
5682865
4372862
3079746
2891875
2891809
2609516


# Get Variables

In [8]:
dtype = [('run', int), 
         ('event', int), 
         ('subevent', int),
        ('logE', float), 
        ('spMPE', float), 
         ('spMPE_rlogl', float), 
         ('spMPE_ldirE', float), 
         ('spMPE_ndirE', float),
        ('spMPE_nearlyE',float),
          ('millipede', float),  
         ('SPEFit2BayesianFitParams_logl', float),
         ('SPEFit2_TWHVFitParams_logl', float),
         ('LineFit_TWHV_azi', float),
         ('LineFit_TWHV_zen', float),
         ('MPEFitHighNoise_azi', float),
         ('MPEFitHighNoise_zen', float),
         ('MPEFit_TWHV_azi', float),
         ('MPEFit_TWHV_zen', float),
         ('cog_z', float),
         ('LineFit2GeoSplit1_zen', float),
         ('LineFit2GeoSplit2_zen', float),
         ('LineFit2TimeSplit1_zen', float),
         ('LineFit2TimeSplit2_zen', float),
         ('spMPE_azimuth', float), 
         ('spMPE_zenith', float), 
         ('MuExrllt',float), 
         ('time', float), ('trueRa', float), ('trueDec', float),('spMPE_parabo', float),
         ('cog_r2', float),('space_angle', float),('LineFit_delta_angle', float),
         ('MPEHighNoise_delta_angle', float),('SplitMinZenith', float),('BayesLLHRatio', float),
         ('ow', float), ('trueE',float),('trueAzi', float), ('trueZen', float),('weights',float),('NEarlyNCh', float),
         ('mc_azimuth', float), 
         ('mc_zenith', float),
         
         
        ]


In [9]:
Variables=[]
Variables+=[('run','I3EventHeader','Run')]
Variables+=[('event','I3EventHeader','Event')]
Variables+=[('subevent','I3EventHeader','SubEvent')]
Variables+=[('logE','SplineMPEMuEXDifferential','energy')]
Variables+=[('spMPE','SplineMPE','fit_status')]
Variables+=[('spMPE_rlogl','SplineMPEFitParams','rlogl')]
Variables+=[('spMPE_ldirE','SplineMPEDirectHitsE','dir_track_length')]
Variables+=[('spMPE_ndirE','SplineMPEDirectHitsE','n_dir_pulses')]
Variables+=[('spMPE_nearlyE','SplineMPEDirectHitsE','n_early_pulses')]
Variables+=[('millipede', 'SplineMPE_MillipedeHighEnergyMIEFitParams', 'exists')] 
Variables+=[('SPEFit2BayesianFitParams_logl', 'SPEFit2BayesianFitParams','logl')]
Variables+=[('SPEFit2_TWHVFitParams_logl', 'SPEFit2_TWHVFitParams','logl')]
Variables+=[('LineFit_TWHV_azi','LineFit_TWHV','azimuth')]
Variables+=[('LineFit_TWHV_zen', 'LineFit_TWHV','zenith')]
Variables+=[('MPEFitHighNoise_azi', 'MPEFitHighNoise','azimuth')]
Variables+=[('MPEFitHighNoise_zen', 'MPEFitHighNoise','zenith')]
Variables+=[('MPEFit_TWHV_azi','MPEFit_TWHV','azimuth')]
Variables+=[('MPEFit_TWHV_zen','MPEFit_TWHV','zenith')]
Variables+=[('cog_z', 'HitStatisticsValues','cog_z')]
Variables+=[('LineFit2GeoSplit1_zen', 'LineFitGeoSplit1','zenith')]
Variables+=[('LineFit2GeoSplit2_zen', 'LineFitGeoSplit2','zenith')]
Variables+=[('LineFit2TimeSplit1_zen', 'LineFitTimeSplit1','zenith')]
Variables+=[('LineFit2TimeSplit2_zen', 'LineFitTimeSplit2','zenith')]
Variables+=[('spMPE_azimuth','SplineMPE','azimuth')]
Variables+=[('spMPE_zenith', 'SplineMPE','zenith')]
Variables+=[('MuExrllt','MuEXAngular4_rllt','value')]



In [10]:
def get_variables(file, Variables,Indicies):
        Array = np.zeros(Indicies.sum(), dtype=dtype)
        for var in Variables:
               Array[var[0]]=file.root._f_get_child(var[1]).read()[var[2]][Indicies]
                
        return Array

In [11]:
Data_train = get_variables(Data_train_file, Variables,Data_train_cut_indices)
Data_test = get_variables(Data_test_file, Variables,Data_test_cut_indices )

In [12]:
def complete_data(Data, Data_file, Data_cut):
    mjd_sec = Data_file.root.I3EventHeader.cols.time_start_mjd_sec[:][Data_cut]
    mjd_ns = Data_file.root.I3EventHeader.cols.time_start_mjd_ns[:][Data_cut]
    mjd_day = ( mjd_sec + mjd_ns*1E-9 )/ 86400.0 + Data_file.root.I3EventHeader.cols.time_start_mjd_day[:][Data_cut]
    Data['time'] = mjd_day
    
    exp_spMPE_parabo_err1 = Data_file.root.MPEFitParaboloidFitParams.cols.err1[:][Data_cut]
    exp_spMPE_parabo_err2 = Data_file.root.MPEFitParaboloidFitParams.cols.err2[:][Data_cut]
    Data['spMPE_parabo']=np.sqrt((exp_spMPE_parabo_err1**2+exp_spMPE_parabo_err2**2)/2)
    Data['cog_r2'] = (Data_file.root.HitStatisticsValues.cols.cog_x[:][Data_cut])**2 + (Data_file.root.HitStatisticsValues.cols.cog_y[:][Data_cut])**2
    
    Data['LineFit_delta_angle'] = delta_angle(Data['spMPE_zenith'],Data['spMPE_azimuth'],
                                              Data['LineFit_TWHV_zen'],Data['LineFit_TWHV_azi'])

    Data['MPEHighNoise_delta_angle'] = delta_angle(Data['MPEFitHighNoise_zen'],Data['MPEFitHighNoise_azi'],
                                                 Data['MPEFit_TWHV_zen'],Data['MPEFit_TWHV_azi'])
    Data['SplitMinZenith'] = SplitMinZenithFunc(Data['LineFit2GeoSplit1_zen'],Data['LineFit2GeoSplit2_zen'],
                                               Data['LineFit2TimeSplit1_zen'],Data['LineFit2TimeSplit2_zen'])
    Data['BayesLLHRatio'] = BayesianFunc(Data['SPEFit2BayesianFitParams_logl'], 
                                         Data['SPEFit2_TWHVFitParams_logl'])
    Data['NEarlyNCh'] = Data['spMPE_nearlyE'] / Data['spMPE_ndirE']
    Data['logE'] = np.log10(Data['logE'])
    
    return Data

In [13]:
Data_train = complete_data(Data_train, Data_train_file, Data_train_cut_indices)
Data_test = complete_data(Data_test, Data_test_file, Data_test_cut_indices)

In [14]:
MC = get_variables(MC_file, Variables,MC_cut_indices)

MC['mc_azimuth'] = MC_file.root.MCMostEnergeticTrack.cols.azimuth[:][MC_cut_indices]
MC['mc_zenith'] = MC_file.root.MCMostEnergeticTrack.cols.zenith[:][MC_cut_indices]
MC['time'] = MC_file.root.I3EventHeader.cols.time_start_mjd[:][MC_cut_indices]
MC['trueAzi'] = MC_file.root.I3MCWeightDict.cols.PrimaryNeutrinoAzimuth[:][MC_cut_indices]
MC['trueZen'] = MC_file.root.I3MCWeightDict.cols.PrimaryNeutrinoZenith[:][MC_cut_indices]
MC['trueRa'], MC['trueDec'] = astro.dir_to_equa(MC['trueZen'], MC['trueAzi'], MC['time'])
sim_spMPE_parabo_err1 = MC_file.root.MPEFitParaboloidFitParams.cols.err1[:][MC_cut_indices]
sim_spMPE_parabo_err2 = MC_file.root.MPEFitParaboloidFitParams.cols.err2[:][MC_cut_indices]
MC['spMPE_parabo'] = np.sqrt((sim_spMPE_parabo_err1**2+sim_spMPE_parabo_err2**2)/2)
MC['cog_r2'] = (MC_file.root.HitStatisticsValues.cols.cog_x[:][MC_cut_indices])**2 + (MC_file.root.HitStatisticsValues.cols.cog_y[:][MC_cut_indices])**2
MC['space_angle']=delta_angle(MC['mc_zenith'],MC['mc_azimuth'],MC['spMPE_zenith'],MC['spMPE_azimuth'])

MC['LineFit_delta_angle'] = delta_angle(MC['spMPE_zenith'],MC['spMPE_azimuth'],MC['LineFit_TWHV_zen'],
                                        MC['LineFit_TWHV_azi'])

MC['MPEHighNoise_delta_angle'] = delta_angle(MC['MPEFitHighNoise_zen'],MC['MPEFitHighNoise_azi'],
                                             MC['MPEFit_TWHV_zen'],MC['MPEFit_TWHV_azi'])

MC['SplitMinZenith'] = SplitMinZenithFunc(MC['LineFit2GeoSplit1_zen'],MC['LineFit2GeoSplit2_zen'],
                                           MC['LineFit2TimeSplit1_zen'],MC['LineFit2TimeSplit2_zen'])
MC['BayesLLHRatio'] = BayesianFunc(MC['SPEFit2BayesianFitParams_logl'], MC['SPEFit2_TWHVFitParams_logl'])

MC['ow'] = MC_file.root.I3MCWeightDict.cols.OneWeight[:][MC_cut_indices]
MC['trueE'] = MC_file.root.I3MCWeightDict.cols.PrimaryNeutrinoEnergy[:][MC_cut_indices]
MC['NEarlyNCh'] = MC['spMPE_nearlyE'] / MC['spMPE_ndirE']

MC['logE'] = np.log10(MC['logE'])

nfiles_sim = 3000
test_hdf=tables.open_file('/data/user/xk35/BDT/hdf_mc_southern_sky/0/Level3_IC86.2016_NuMu.021002.001999.hdf')
processed_events = test_hdf.root.I3MCWeightDict.cols.NEvents[:][0]


MC['ow'] = MC['ow']/(nfiles_sim*processed_events)


LT_day = 0
for run in np.unique(Data_train['run']):
    mask = (Data_train['run'] == run)
    min_time = np.min(Data_train[mask]['time'])
    max_time = np.max(Data_train[mask]['time'])
    run_time = max_time - min_time
    
    LT_day += run_time

LT = LT_day*86400

print(LT_day)

fluxNorm = 1.0e-18
MC['weights'] = fluxNorm * MC['ow'] * (MC['trueE']/1.0e5)**(-gamma) * LT

         
         

23.808155883962172


In [25]:
#np.save('post_precut_datasets/MC_all_aky.npy',MC)
#np.save('post_precut_datasets/Data_test_all_aky.npy',Data_test)
#np.save('post_precut_datasets/Data_train_all_sky.npy',Data_train)